In [45]:
## imporrt libarary 
import os
import pandas as pd

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
# from google.cloud import aiplatform
# from support_functions import missing_value, fill_missing, list_dtypes

## sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pickle


In [46]:
#pip install google-cloud-bigquery

In [47]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ""

In [48]:
#client = bigquery.Client('dtidsUS')

In [49]:
#query_job = client.query(f"""select * from {dataset_id}.{table_id}""")

In [50]:
project_id = 'dtidsUS'
dataset_id = 'capstone'
table_id = 'data_bank_marketing_campaign'
region = 'us-central1'
bucket_name = 'modul4'
blob_name = 'ilham/data_bank_marketing_campaign.csv'

In [51]:
client = bigquery.Client(project='dtidsus')

#uplad to google cloud

In [52]:
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    data_capstone = bucket.blob('ilham/data_bank_marketing_campaign.csv')
    data_capstone.upload_from_filename('data_bank_marketing_campaign.csv')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

TypeError: An exception occurred

In [53]:
client = bigquery.Client('dtidsUS')

In [54]:
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")

BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/dtidsUS/jobs?prettyPrint=false: ProjectId must be non-empty

Location: None
Job ID: 1838eedc-be06-45c1-9ed2-a25f9b45f72f


In [55]:
df = query_job.result().to_dataframe()

NameError: name 'query_job' is not defined

In [15]:




#cleansing 
result = df.drop(['ID'], axis = 1)
result.head()

,LIMIT_BAL,MARRIAGE,EDUCATION,SEX,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,TARGET
0,50000,married,Univ,male,23,131895,132154,129237,1
1,50000,married,Univ,male,24,56626,58512,61926,1
2,40000,married,Univ,male,25,12961,12029,11738,1
3,30000,single,Univ,male,26,28949,30319,29400,0
4,30000,married,Univ,male,26,0,0,0,1


In [17]:
result.isna().sum()

LIMIT_BAL    0
MARRIAGE     4
EDUCATION    0
SEX          0
AGE          4
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
TARGET       0
dtype: int64

In [19]:
result  = result.dropna()

In [20]:
categorical_cols = result.select_dtypes(include=['object']).columns

# Create a dictionary to store label encoders for each column
encoders = {}

# Iterate through categorical columns
for col in categorical_cols:
    # Create a LabelEncoder object for each column
    encoder = LabelEncoder()

    # Fit and transform the column
    result[col] = encoder.fit_transform(result[col])

    # Store the encoder in the dictionary for later use
    encoders[col] = encoder


# Preprocessing
y = result['TARGET']  
X = result.drop(['TARGET'], axis=1)  

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Evaluation
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

# Save Model
model_filename = "model.pkl"
pickle.dump(model, open(model_filename, 'wb'))

              precision    recall  f1-score   support

         0.0       0.68      0.76      0.72      1980
         1.0       0.58      0.48      0.52      1344

    accuracy                           0.65      3324
   macro avg       0.63      0.62      0.62      3324
weighted avg       0.64      0.65      0.64      3324



c:\Users\cbcbx\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##upload the model

In [ ]:
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob('ilham/model/model.pkl')
    blob_model.upload_from_filename('model.pkl')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [32]:
from google.cloud import aiplatform

In [33]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "dev_trial.json"

In [35]:
aiplatform.init(project='dtidsus', location='us-central1')

model = aiplatform.Model.upload(
    display_name='jaya_model_000',
    artifact_uri="gs://modul4/ilham/model/",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest",
)

model.wait()

Creating Model
Create Model backing LRO: projects/41965541199/locations/us-central1/models/4379281146152747008/operations/8350858329844613120
Model created. Resource name: projects/41965541199/locations/us-central1/models/4379281146152747008@1
To use this Model in another session:
model = aiplatform.Model('projects/41965541199/locations/us-central1/models/4379281146152747008@1')


In [36]:
endpoint = aiplatform.Endpoint.create(
    display_name="jaya-endpoint-000",
    project='dtidsus',
    location='us-central1',
)

Creating Endpoint
Create Endpoint backing LRO: projects/41965541199/locations/us-central1/endpoints/2241324766407426048/operations/8402649725559373824
Endpoint created. Resource name: projects/41965541199/locations/us-central1/endpoints/2241324766407426048
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/41965541199/locations/us-central1/endpoints/2241324766407426048')


In [39]:
min_replica_count: int = 1
max_replica_count: int = 1

In [40]:
endpoint.deploy( 
    model=model,
    deployed_model_display_name='jaya_model_000',
    machine_type='e2-standard-2',
    min_replica_count=min_replica_count,
    max_replica_count=max_replica_count,
    sync=True, 
)

Deploying Model projects/41965541199/locations/us-central1/models/4379281146152747008 to Endpoint : projects/41965541199/locations/us-central1/endpoints/2241324766407426048
Deploy Endpoint model backing LRO: projects/41965541199/locations/us-central1/endpoints/2241324766407426048/operations/54664866274738176
Endpoint model deployed. Resource name: projects/41965541199/locations/us-central1/endpoints/2241324766407426048


#using endpoint

In [45]:
## predict your data with online prediction here 
PROJECT_ID = 'dtidsus'
ENDPOINT_ID = "projects/41965541199/locations/us-central1/endpoints/2241324766407426048"
REGION = 'us-central1'



In [46]:
aiplatform.init(project=PROJECT_ID, location=REGION)
endpoint = aiplatform.Endpoint(ENDPOINT_ID)
prediction = endpoint.predict(instances=[[50000, 1, 3, 1, 23, 131895, 132154, 129237]])

print("PREDICTION:", prediction)

PREDICTION: Prediction(predictions=[0.0], deployed_model_id='6724688981780332544', metadata=None, model_version_id='1', model_resource_name='projects/41965541199/locations/us-central1/models/4379281146152747008', explanations=None)


#manual file

In [28]:
try:
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob_model = bucket.blob('ilham/model/model.pkl')
    blob_model.download_to_filename('model_new.pkl')

    print ("download model succeeded")
except:
    raise TypeError("An exception occurred")

download model succeeded


In [29]:
model_load = pickle.load(open('model_new.pkl','rb'))

In [30]:
y_predict = model_load.predict(X_test)

In [31]:
y_predict

array([0., 0., 0., ..., 0., 0., 1.])